In [ ]:
import serial
import time
import numpy as np
import cv2
import tensorflow as tf


In [ ]:
# Load trained model
model = tf.keras.models.load_model("thumbs_up_model.h5")
img_size = (128, 128)

# Connect to Arduino
ser = serial.Serial('/dev/cu.usbmodem1301', 115200, timeout=5)
time.sleep(2)

# Wait for camera to start
while True:
    line = ser.readline().decode('utf-8', errors='ignore').strip()
    if line == "START":
        print("Image capture started.")
        break

# Receive JPEG data
jpeg_data = b''
while True:
    chunk = ser.read(1024)
    if b'\xff\xd9' in chunk:
        jpeg_data += chunk[:chunk.find(b'\xff\xd9') + 2]
        break
    jpeg_data += chunk

# Decode image
image_np = np.frombuffer(jpeg_data, dtype=np.uint8)
img = cv2.imdecode(image_np, cv2.IMREAD_COLOR)
cv2.imwrite("captured.jpg", img)


In [ ]:
# Predict thumbs up
img_resized = cv2.resize(img, img_size) / 255.0
img_tensor = np.expand_dims(img_resized, axis=0)
prediction = model.predict(img_tensor)[0][0]

if prediction > 0.5:
    print("✅ Thumbs Up Detected!")
    ser.write(b'ROTATE\n')
else:
    print("❌ Not a Thumbs Up")